In [33]:
import numpy as np
import sympy as sym
from IPython.display import Math, Latex
from IPython.display import Image
import sympy as sp
import numpy as np
from scipy.integrate import quad
import pandas as pd
import matplotlib.pyplot as plt

In [34]:
e11 = np.array([])
u11 = np.array([])
u12 = np.array([])
NU12 = np.array([])
# Create an empty DataFrame with the specified columns
df = pd.DataFrame(columns=['H', 'L', 'h_rib', 't', 'u11', 'u12', 'nu12', 'e11'])

# Define variables and symbolic expressions
L, H, t, w = sp.symbols('L H t w')
theta = sp.symbols(r'\theta')
A_eff1, A_eff2 = sp.symbols(r'A_\mathrm{eff_{1}}, A_\mathrm{eff_{2}}')
h_rib = sp.Symbol(r'h_\mathrm{rib}')
A = sp.Symbol(r'A')
I = sp.Symbol(r'I')
U = sp.Symbol(r'U')
E = sp.Symbol(r'E')
x, y, s = sp.symbols('x y s')
Mf, Nf, Vf, P, Q = sp.symbols(r'M_\mathrm{f} N_\mathrm{f} V_\mathrm{f} P Q')
pi = sp.Symbol(r'\pi')
sigma = sp.Symbol(r'\sigma')
B = sp.Symbol('B')

# Substitutions (remove t, h_rib, P from known values)
subs = {
    I: (w * t**3) / 12,
    A: w * t,
    A_eff1: (2 * H - 2 * h_rib) * w,
    A_eff2: 2 * L * w,
    B: 2 * L
}

# Known constants (without t, h_rib, and P)
known_values = {
    w: 1,
    pi: 3.14159,
    E: 69,
}

# Define vertical displacement y
y = (h_rib / 2) * (1 - sp.cos(pi * x / L))

# Derivative of y with respect to x
d_x_coeff = sp.sqrt(1 + (sp.diff(y, x))**2)

# Bending moment Mf11
Mf11 = ( P*h_rib / 2) -  P*y
Mf11 = Mf11.simplify()

# U_M coefficient for Mf11 (moment)
UM11_coeff = (Mf11**2 / (2 * E * I)) * d_x_coeff
UM11_coeff_simplified = sp.simplify(UM11_coeff)
UM11_coeff_with_vars = UM11_coeff_simplified.subs(subs)

# --- Normal force energy calculation (U_N) ---

# Define the normal force Nf_11
theta = sp.atan(sp.diff(y, x))
Nf_11_expr =  P*sp.cos(theta)
Nf_11_simplified = Nf_11_expr.simplify()

# U_N coefficient for Nf11 (normal force)
UN11_coeff = (Nf_11_simplified**2 / (2 * E * A)) * d_x_coeff
UN11_coeff_simplified = sp.simplify(UN11_coeff)
UN11_coeff_with_vars = UN11_coeff_simplified.subs(subs)

# --- Bending moment (Mf12) ---
Mf12 = Q * x + (P * y - (P * h_rib / 2))
Mf12 = Mf12.simplify()

# U_M coefficient for Mf12 (moment)
UM12_coeff = (Mf12**2 / (2 * E * I)) * d_x_coeff
UM12_coeff_simplified = sp.simplify(UM12_coeff)
UM12_coeff_with_vars = UM12_coeff_simplified.subs(subs)

# --- Normal force energy calculation (Nf12) ---
theta = sp.atan(sp.diff(y, x))
Nf12_expr = P * sym.cos(theta) - Q * sym.sin(theta)
Nf12_simplified = Nf12_expr.simplify()

# U_N coefficient for Nf12 (normal force)
UN12_coeff = (Nf12_simplified**2 / (2 * E * A)) * d_x_coeff
UN12_coeff_simplified = sp.simplify(UN12_coeff)
UN12_coeff_with_vars = UN12_coeff_simplified.subs(subs)


# --- Loop to vary t, h_rib, and P ---
t_values = np.linspace(0.8, 1.8, 20)   # Example values for t
#t_values = [1.2]
#h_rib_values = np.linspace(0.001, 4, 20)    # Example values for h_rib
#h_rib_values = [3]
P_values = [1]      # Example values for P
# Define the range of values for H and B
H_values = np.linspace(6,15,20)   # Example values for H
#H_values = [10]
l_values = np.linspace(6,15,20)  # Example values for L
#l_values = [10]
# Perform integration for different values of H, B, t, h_rib, and P
for H_val in H_values:
    for l_val in l_values:
        for t_val in t_values:
            for P_val in P_values:
                h_rib_max  = ((H_val/2)-t_val)
                h_rib_values = np.linspace(0.001, h_rib_max, 20)
                for h_rib_val in h_rib_values:
                    # Create specific substitutions for this iteration
                    variable_values = {
                        t: t_val,
                        h_rib: h_rib_val,
                        H: H_val,
                        L: l_val,
                    }
                    
                    # --- Energy due to bending moment (U_M) ---
                    UM11_coeff_substituted = UM11_coeff_with_vars.subs({**known_values, **variable_values})
                    UM11_diff = sym.diff(UM11_coeff_substituted, P)
                    UM11_diff_substituted = UM11_diff.subs(P, P_val)
                    UM11_numeric = sp.lambdify(x, UM11_diff_substituted, 'numpy')
                    UM11_result, UM11_error = quad(UM11_numeric, 0, variable_values[L])

                    # --- Energy due to normal force (U_N) ---
                    UN11_coeff_substituted = UN11_coeff_with_vars.subs({**known_values, **variable_values})
                    UN11_diff = sym.diff(UN11_coeff_substituted, P)
                    UN11_diff_substituted = UN11_diff.subs(P, P_val)
                    UN11_numeric = sp.lambdify(x, UN11_diff_substituted, 'numpy')
                    UN11_result, UN11_error = quad(UN11_numeric, 0, variable_values[L])

                    u1_1_diff = UN11_result + UM11_result
                    u11 = np.append(u11, u1_1_diff)
                   
                    Es = 69
                    sigma = 1/A_eff1
                    E11_exp = (sigma*B)/(u1_1_diff)
                    A_eff1_value = subs[A_eff1].subs({**known_values, **variable_values})
                    B_value = subs[B].subs({**known_values, **variable_values})

                    # Substitute A_eff1 and B in E11_exp
                    E11_exp_with_values = E11_exp.subs({A_eff1: A_eff1_value, B: B_value, **known_values, **variable_values})

                    # Evaluate the result numerically
                    E11_result = float(E11_exp_with_values / Es)  # Es is the modulus

                    # --- Energy due to bending moment (U_M) for Mf12 ---
                    UM12_coeff_substituted = UM12_coeff_with_vars.subs({**known_values, **variable_values})
                    UM12_diff = sym.diff(UM12_coeff_substituted, Q)
                    UM12_diff_substituted = UM12_diff.subs({Q: 0, P: P_val})
                    UM12_numeric = sp.lambdify(x, UM12_diff_substituted, 'numpy')
                    #print(UM12_diff_substituted)
                    UM12_result, UM12_error = quad(UM12_numeric, 0, variable_values[L])

                    # --- Energy due to normal force (U_N) for Nf12 ---
                    UN12_coeff_substituted = UN12_coeff_with_vars.subs({**known_values, **variable_values})
                    UN12_diff = sym.diff(UN12_coeff_substituted, Q)
                    UN12_diff_substituted = UN12_diff.subs({Q: 0, P: P_val})
                    UN12_numeric = sp.lambdify(x, UN12_diff_substituted, 'numpy')

                    UN12_result, UN12_error = quad(UN12_numeric, 0, variable_values[L])

                    u12_diff = UN12_result + UM12_result
                    u12 = np.append(u12, u12_diff)

                    # Calculate nu12
                    nu12 = -((u12_diff*2*(l_val))/((2*u1_1_diff )*(H_val - h_rib_val)))
                    NU12 = np.append(NU12, nu12)

                    # Create a dictionary with the variables
                    row_data = {
                        'H': H_val,
                        'L': l_val,
                        'h_rib': h_rib_val,
                        't': t_val,
                        'u11': u1_1_diff,
                        'u12': u12_diff,
                        'nu12': nu12,
                        'e11': E11_result,
                    }

                    # Convert the row data to a DataFrame and append it
                    row_df = pd.DataFrame([row_data])
                    df = pd.concat([df, row_df], ignore_index=True)


C:\Users\Ayush Dubey\AppData\Local\Temp\ipykernel_4596\1075222642.py:179: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, row_df], ignore_index=True)
